# Task 1

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:

data_path = "../../data/dataset/sample/"

ap = pd.read_csv(data_path+'/anagraficapazientiattivi.csv', header=0 ,names=['idcentro','idana','sesso','annodiagnosidiabete','tipodiabete','scolarita','statocivile','professione','origine','annonascita','annoprimoaccesso','annodecesso'])
diag = pd.read_csv(data_path+'/diagnosi.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
elp  = pd.read_csv(data_path+'/esamilaboratorioparametri.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
ei = pd.read_csv(data_path+'/esamistrumentali.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pdf = pd.read_csv(data_path+'/prescrizionidiabetefarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceatc','quantita','idpasto','descrizionefarmaco'])
pdnf = pd.read_csv(data_path+'/prescrizionidiabetenonfarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pnd = pd.read_csv(data_path+'/prescrizioninondiabete.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])

In [ ]:
# Display example observations
ap.head()

In [ ]:
diag.head()

In [ ]:
elp.head()

In [ ]:
ei.head()

In [ ]:
pdf.head()

In [ ]:
pdnf.head()

In [ ]:
pnd.head()

Borrar pacientes que:
  - fecha nacimiento > fecha primo accessso || fecha nacimiento > fecha muerte || fecha nacimiento > fecha diagnostico
  - fecha primo accesso > fecha muerte || fecha primo accesso > fecha diagnostico
  - fecha diagnostico > fecha muerte

Resto tablas, borrar pacientes que:
  - fecha < fecha nacimiento || fecha < fecha primo accesso || fecha > fecha muerte

En tablas de medicamentos:
  - fecha entre fecha diagnostico y fecha muerte

First we want to select from our tables the patients with dates that are valid (with the birth year smaller than the death year, the birth year smaller than the year of first access and so on)

In [ ]:
ap.shape

In [ ]:
def exercise2_1 (df) -> pd.DataFrame:
    aux = df[(df['annonascita'].notna()) & (df['annoprimoaccesso'].notna()) & (df['annodiagnosidiabete'].notna()) & (df['annonascita']<=df['annodecesso']) & (df['annoprimoaccesso']<=df['annodecesso']) & (df['annodiagnosidiabete']<=df['annodecesso'])]
    aux = pd.concat([aux, df[df['annodecesso'].isna()]])
    return aux[(aux['annonascita']<=aux['annodiagnosidiabete']) & (aux['annonascita']<=aux['annoprimoaccesso'])]

ap_ex2 = exercise2_1(ap)
print(ap_ex2.shape)    

In [ ]:
# Let's ensure that there aren't duplicate rows:
ap_test = ap_ex2.filter(items=['idcentro', 'idana'])
print(ap_test.shape)
ap_test.drop_duplicates()
print(ap_test.shape)

In [ ]:
def exercise2_2(df) -> pd.DataFrame:
    initial_columns = df.columns
    # First we eliminate the patients that are no longer in the first table and the dates that do not exist
    df = df.merge(ap_test, on=['idcentro', 'idana'], how='inner')
    df = df[df['data'].notna()]

    # We assure that the dates are valid and that the year is first
    df['data'] = pd.to_datetime(df['data'], yearfirst=True)

    #Now we create a new table with the dates from the first and the second table
    ap_filter = ap_ex2.filter(items=['idcentro', 'idana', 'annonascita', 'annoprimoaccesso', 'annodecesso'])
    aux = df.merge(ap_filter, on=['idcentro', 'idana'])

    # Now we take the rows of this table that have valid dates
    aux = aux[aux['data'].dt.strftime("%Y").astype(float) <= aux['annodecesso']]
    aux = pd.concat([aux, ap_filter[ap_filter['annodecesso'].isna()]])
    aux = aux[(aux['data'].dt.strftime("%Y").astype(float) >= aux['annoprimoaccesso']) & (aux['data'].dt.strftime("%Y").astype(float) >= aux['annonascita'])]

    if 'annodiagnosidiabete' in df.columns:
        aux = aux[(aux['data'].dt.strftime("%Y").astype(float) >= aux['annodiagnosidiabete'])]

    # Now we take into account only the new diagnosis but we need to delete the extra columns
    return aux.filter(df.columns)



In [ ]:
diag_ex2 = exercise2_2(diag)
print("Diag:",diag_ex2.shape)

elp_ex2 = exercise2_2(elp)
print("Elp:",elp_ex2.shape)

ei_ex2 = exercise2_2(ei)
print("Ei:",ei_ex2.shape)

pdf_ex2 = exercise2_2(pdf)
print("Pdf:",pdf_ex2.shape)

pdnf_ex2 = exercise2_2(pdnf)
print("Pdnf:",pdnf_ex2.shape)

pnd_ex2 = exercise2_2(pnd)
print("Pdnf:",pnd_ex2.shape)